## EDA on NYC Taxi data

In [1]:
# Import required packages
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DoubleType
import pyspark.sql.functions as F

In [2]:
# Create a local spark session
spark = SparkSession.builder \
        .appName('nyc-taxi-data') \
        .getOrCreate()

In [3]:
# Set parameters 
bucket_name = "nyc-tlc" # s3 bucket name with required nyc tlc files

In [4]:
# Create function to read S3 bucket
def list_bucket_contents(bucket, match=''):
    files = []
    s3_resource = boto3.resource('s3')
    bucket_resource = s3_resource.Bucket(bucket)
    for key in bucket_resource.objects.all():
        if match in key.key:
            files.append(key.key)
    return files

In [5]:
# Read yellow and green taxi data for respective years
colours = ["yellow","green"]
years = ["2019","2020"]
files = []

for year in years:
    for colour in colours:
        match = colour + "_tripdata_" + year
        files.extend(list_bucket_contents(bucket=bucket_name, match=match))

In [8]:
files

['trip data/yellow_tripdata_2019-01.csv',
 'trip data/yellow_tripdata_2019-02.csv',
 'trip data/yellow_tripdata_2019-03.csv',
 'trip data/yellow_tripdata_2019-04.csv',
 'trip data/yellow_tripdata_2019-05.csv',
 'trip data/yellow_tripdata_2019-06.csv',
 'trip data/yellow_tripdata_2019-07.csv',
 'trip data/yellow_tripdata_2019-08.csv',
 'trip data/yellow_tripdata_2019-09.csv',
 'trip data/yellow_tripdata_2019-10.csv',
 'trip data/yellow_tripdata_2019-11.csv',
 'trip data/yellow_tripdata_2019-12.csv',
 'trip data/green_tripdata_2019-01.csv',
 'trip data/green_tripdata_2019-02.csv',
 'trip data/green_tripdata_2019-03.csv',
 'trip data/green_tripdata_2019-04.csv',
 'trip data/green_tripdata_2019-05.csv',
 'trip data/green_tripdata_2019-06.csv',
 'trip data/green_tripdata_2019-07.csv',
 'trip data/green_tripdata_2019-08.csv',
 'trip data/green_tripdata_2019-09.csv',
 'trip data/green_tripdata_2019-10.csv',
 'trip data/green_tripdata_2019-11.csv',
 'trip data/green_tripdata_2019-12.csv',
 'tr

In [6]:
# Read January 2019 yellow taxi cab data from S3 bucket
yellow_df = spark.read.csv(f"s3a://{bucket_name}/trip data/yellow_tripdata_2019-01.csv", header=True)

In [9]:
# Show first twenty rows of the imported file
yellow_df.show(20)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|              1|         1.50|         1|                 N|         151|         239|           1|          7|  0.5|    0.5|      1.65|           0|                  0.3

In [13]:
# Print schema of data frame to show field data types and nullability
yellow_df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



In [10]:
# Read January 2019 green taxi cab data from S3 bucket
green_df = spark.read.csv(f"s3a://{bucket_name}/trip data/green_tripdata_2019-01.csv", header=True)

In [12]:
green_df.show(10)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2018-12-21 15:17:29|  2018-12-21 15:18:57|                 N|         1|         264|         264|              5|          .00|          3|  0.5|    0.

In [14]:
# Print schema of data frame to show field data types and nullability
green_df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



## Recommended data type changes

All fields imported as a string. The following data type conversions are required:

* VendorID: string -> categorical (drop - needs processing intensive one hot encoding)
* lpep_pickup_datetime: string -> timestamp
* lpep_dropoff_datetime: string -> timestamp
* store_and_fwd_flag: string -> integer (drop)
* RatecodeID: string -> integer
* PULocationID: string -> categorical (drop - needs processing intensive one hot encoding)
* DOLocationID: string -> categorical (drop - needs processing intensive one hot encoding)
* passenger_count: string -> integer
* trip_distance: string -> double
* fare_amount: string -> double
* extra: string -> double
* mta_tax: string -> double (drop - is always $0.50)
* tip_amount: string -> double
* tolls_amount: string -> double
* ehail_fee: string -> double (drop - is set fee)
* improvement_surcharge: string -> double (drop - is set fee)
* total_amount: string -> double
* payment_type: string -> integer
* trip_type: string -> integer (drop - is not in other data set)
* congestion_surcharge: string -> double (drop - is not in other data set and should be included in extras per dictionary)